# Neural Ordinary Differential Equations
A discussion of an interesting new paradigm in Neural Networks, adapted from "Neural Ordinary Differential Equations" by Chen et al. 2019 (https://arxiv.org/pdf/1806.07366.pdf)

## 1. Introduction
### 1.1 - Very Brief Overview of Neural Networks
Neural networks are a set of algorithms that are loosely based on the functonality of the human brain. These algorithms are designed to learn patterns in numerical data, by propagating input data through a series of hidden layers, with each layer being composed of several artificial 'neurons' or nodes. The final result is the output layer, a numerical representation of our desired prediction.

Consider an input dataset composed of a set of $m$ features: $(\vec{x_1}, \vec{x_2}, ..., \vec{x_m})$ . <br>
We add a bias component to this dataset, $\vec{x_0} = 1$, a vector of ones, yielding our input layer, $\vec{x} := \vec{x_0}, ... \vec{x_m}$. <br>
We need to now consider how to pass this input to our first hidden layer. Suppose this the first layer is composed of $n$ nodes, we define a set of weights $w=\vec{w_1},\vec{w_2} ..., \vec{w_n}$ where each $\vec{w_i}$ is a vector of length $m$ corresponding to node $i$. We then define $\vec{z_i} := \vec{x}\cdot \vec{w_{i}}$. <br>

We are passing $\vec{z_i}$ to each node in the first hidden layer, but but what occurs at the node, and what is output at the next layer? The answer is the activation function. For the purpose of this discussion, we will focus solely on the sigmoid activation function $\sigma(z) = \dfrac{1}{1+e^{-z}}$, though it should be noted that many others exist. At each node, we will compute $\sigma(z_i)$ and that's it! We have our single layer representation. <br>

To extrapolate to a full neural network, we simply repeat this process, using the output of the activation functions of the previous layer as the new input layer, taking the dot product with a new set of weights and computing the activation of these new inputs. The final layer will be our output layer, $\vec{y}$, and will have a number of neurons corresponding to our desired predictable (if we are predicting hand-written digits as in the exemplary MNIST dataset, our $\vec{y}$ will be a vector of length 10). <br>

Our problem is thus clear, we must compute: a) the weights and b) the number of neurons in each layer. And we must do so efficiently, as there will often be many many weights to compute and many layers in our network. 

In [ ]:
import os
import pandas as pd
import numpy as np